<a href="https://colab.research.google.com/github/Psyhoved/MTS_chatbot/blob/main/AI_chat_bot_for_MTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index
!pip install langchain

In [33]:
from langchain import OpenAI
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper, GPTVectorStoreIndex
from llama_index import LLMPredictor, ServiceContext
from llama_index import StorageContext, load_index_from_storage
import sys
from google.colab import drive
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.environ["OPENAI_API_KEY"] = input('Введите ваш API-ключ от Open AI: ')

Создаем index контекста, то есть векторизуем его с помощью библиотек Open AI

In [38]:
def construct_index(directory_path):
  # set maximum input size
  max_input_size = 4096
  # set number of output tokens
  num_outputs = 128
  # set maximum chunk overlap
  max_chunk_overlap = 0.9
  # set chunk size limit
  chunk_size_limit = 600

  prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

  # define LLM
  llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="gpt-3.5-turbo", max_tokens=num_outputs))

  documents = SimpleDirectoryReader(directory_path).load_data()

  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
  index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context, prompt_helper=prompt_helper)

  index.storage_context.persist(persist_dir="index.json")

  return index

Функция для проверки языка ответа, чтобы заставить chatGPT перевести, если он вдруг ответит на английском

In [39]:
import re
def check_russian(text: str) -> bool:
  return bool(re.search('[а-яА-Я]', text))

Основная функция для общения с чат-ботом

In [40]:
def ask_bot(input_index = 'index.json'):
  storage_context = StorageContext.from_defaults(persist_dir=input_index )
  # load index
  index = load_index_from_storage(storage_context)
  memory = 'Здравствуйте, чем я могу вам помочь?'
  print("AI-помощник от МТС: " + memory)
  instruction = 'Ты - чат-бот Светлана, ты девушка, и работаешь на сайте МТС "Мобильные сотрудники", твоя функция - стараться ответить на любой вопрос клиента. Отвечай весело, позитивно, можешь шутить. Используй в ответах только русский язык! Не отвечай на английском! Если вопрос не касается контекста, то вежливо и дружелюбно переведи тему и расскажи про сервис Мобильные сотрудники, МТС Координатор и МТС Трекер! Отвечай на вопросы про сервис Мобильные сотрудники, МТС Координатор и МТС Трекер. Не используй слово "контекст".'
  memory =  instruction + memory
  while True:
    query_engine = index.as_query_engine()
    query = input('Клиент: ')
    response = query_engine.query(memory + query + ' Ответь на последний вопрос или позитивно поддержи диалог')
    if not response.response:
        print("\nAI-помощник от МТС: Не поняла Вашего вопроса. Напишите, пожалуйста, точнее, чтобы Вы хотели узнать.\n")
    elif not check_russian(response.response):
        response = query_engine.query(memory + query + ' Переведи на русский')
        print("\nAI-помощник от МТС: " + response.response + "\n")
        memory = instruction + query + ' ' + response.response
    else:
        print("\nAI-помощник от МТС: " + response.response + "\n")
        memory = instruction + query + ' ' + response.response


In [ ]:
# готовый embedding
!git clone https://github.com/Psyhoved/MTS_chatbot/
!unzip /content/MTS_chatbot/index.json-20230814T192851Z-001.zip

In [42]:
# сделать новый embedding контекста
# !pip install pypdf
# !pip install docx2txt
# drive.mount ('/content/gdrive')
# index = construct_index("/content/gdrive/MyDrive/Colab_Notebooks/docs")

In [ ]:
ask_bot('index.json')